# Hijab Detection Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.model_selection
import tensorflow as tf
import glob
import cv2
import os
from PIL import Image

## Network setup

### VGGFace
Due to being offline I had to download the VGGFace away from this script and simply load it in

```
from keras_vggface.vggface import VGGFace

model = VGGFace(model='resnet50', include_top=False, input_shape=(200, 200, 3), pooling='avg')
model.save('gender_detection/models/vggface')
```

In [2]:
vggface = tf.keras.models.load_model(os.path.expanduser("/home/gridsan/groups/irancovid/models/vggface"), compile=False)
# Make ALL VGGFace layers NOT trainable
for layer in vggface.layers:
    layer.trainable = False
#vggface.summary(line_length=150)

### Adding Hijab Detection Layers

In [3]:
# Create the hijab detection network
hijab_detection = tf.keras.Sequential()

# Add the VGGFace network
hijab_detection.add(vggface)
# And a batch normalization layer
hijab_detection.add(tf.keras.layers.BatchNormalization())
# And finally, a sigmoid output layer
hijab_detection.add(tf.keras.layers.Dense(2, activation="sigmoid"))

# Print the summary
hijab_detection.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_resnet50 (Functional (None, 2048)              23561152  
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,573,442
Trainable params: 8,194
Non-trainable params: 23,565,248
_________________________________________________________________


In [4]:
# Compile the hijab detection network
hijab_detection.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Load in the Data

In [5]:
# NOTE: Check out the Hijab labelling pipeline to see how I labeled it
data = pd.read_csv("/home/gridsan/groups/irancovid/data/hijab-detection/temp/labeled_hijab_images.csv")
# Remove any uncertain labels
data = data[data.hijab != -1]
# And rely on the scaled image for training
data["image_path"] = data["image_path_scaled"]

,image_path_regular,image_path_scaled,image_name,language,hijab,image_path
0,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CF-4lmbnVxP,Arabic,1,/home/gridsan/groups/irancovid/data/hijab-dete...
1,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CGhI8ROACO_,Arabic,1,/home/gridsan/groups/irancovid/data/hijab-dete...
2,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CJytLBbJO90,Arabic,1,/home/gridsan/groups/irancovid/data/hijab-dete...
3,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CFjd3w_An8l,Arabic,1,/home/gridsan/groups/irancovid/data/hijab-dete...
4,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CAtVAbxD1qr,Arabic,1,/home/gridsan/groups/irancovid/data/hijab-dete...
...,...,...,...,...,...,...
12532,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CItTs4rJZ-o,Turkish,1,/home/gridsan/groups/irancovid/data/hijab-dete...
12533,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,B2RYAoOA-Ul,Turkish,0,/home/gridsan/groups/irancovid/data/hijab-dete...
12534,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,CC4BaHlgxRL,Turkish,1,/home/gridsan/groups/irancovid/data/hijab-dete...
12535,/home/gridsan/groups/irancovid/data/hijab-dete...,/home/gridsan/groups/irancovid/data/hijab-dete...,BvwLWz-hQZV,Turkish,1,/home/gridsan/groups/irancovid/data/hijab-dete...


In [6]:
# Split train and test data
imagepaths_train, imagepaths_test, hijab_train, hijab_test = sklearn.model_selection\
    .train_test_split(data[["image_path"]], data[["hijab"]], test_size=0.3, random_state=1234)
    
# Convert outcomes to approriate keras types
hijab_train = tf.keras.utils.to_categorical(hijab_train.hijab.to_numpy())
hijab_test = tf.keras.utils.to_categorical(hijab_test.hijab.to_numpy())

# Load images to np.arrays
images_train = np.asarray(imagepaths_train.image_path\
    .apply(lambda x: tf.keras.preprocessing.image.img_to_array(\
        tf.keras.preprocessing.image.load_img(x, target_size=(200,200))\
    )).to_list())
images_test = np.asarray(imagepaths_test.image_path\
    .apply(lambda x: tf.keras.preprocessing.image.img_to_array(\
        tf.keras.preprocessing.image.load_img(x, target_size=(200,200))\
    )).to_list())

## Train and Evaluate the Model

In [7]:
history = hijab_detection.fit(images_train, hijab_train, batch_size=32, epochs=15, 
    validation_data=(images_test, hijab_test),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights = True)])

Epoch 1/15
248/248 [==============================] - 66s 255ms/step - loss: 0.4899 - accuracy: 0.7745 - val_loss: 0.3011 - val_accuracy: 0.8860
Epoch 2/15
248/248 [==============================] - 54s 220ms/step - loss: 0.2800 - accuracy: 0.8830 - val_loss: 0.2830 - val_accuracy: 0.8928
Epoch 3/15
248/248 [==============================] - 54s 219ms/step - loss: 0.2725 - accuracy: 0.8894 - val_loss: 0.2805 - val_accuracy: 0.8884
Epoch 4/15
248/248 [==============================] - 54s 217ms/step - loss: 0.2461 - accuracy: 0.9030 - val_loss: 0.2858 - val_accuracy: 0.8887
Epoch 5/15
248/248 [==============================] - 53s 215ms/step - loss: 0.2459 - accuracy: 0.9029 - val_loss: 0.2786 - val_accuracy: 0.8937
Epoch 6/15
248/248 [==============================] - 53s 216ms/step - loss: 0.2284 - accuracy: 0.9086 - val_loss: 0.2745 - val_accuracy: 0.8951
Epoch 7/15
248/248 [==============================] - 54s 218ms/step - loss: 0.2317 - accuracy: 0.9047 - val_loss: 0.2954 - val_ac

In [8]:
# And save the model
hijab_detection.save(os.path.expanduser("~/irancovid/models/hijab_detection"))

INFO:tensorflow:Assets written to: /home/gridsan/akapl/irancovid/models/hijab_detection/assets
